# minGemma

our goal here is to teach all of the architecture concepts used in Gemma. This code is a frankenstein mixture between Andrej Karpathy's minGPT and google's pytorch implementation of Gemma. As such, certain things will or won't be included. 


#### Included:
- Grouped Attention
- KV caching
- 

#### Not Included: 
- the ability to change between datatypes (float16 vs float32)
- the full 256,000 token vocabulary of Gemma. Instead we'll be doing character-wise tokenization of TinyShakespeare
- 

In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# config.py
import dataclasses # used for the config. this library is good at making classes that are full of just datatypes
from typing import Optional

# model.py
import re
from typing import Any, List, Sequence, Tuple, Union#, Optional
#from gemma import config as gemma_config
#from gemma import tokenizer

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
#device = 'cpu'

In [9]:
# the dataset we'll be using is just TinyShakespeare
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

# here are all the unique characters that occur in this text. we'll do character-wise tokenization
chars = sorted(list(set(text)))
v = len(chars)
print(chars, v)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [10]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [11]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest validation
train_data = data[:n]
val_data = data[n:]

In [17]:
# data loading for training
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - t, (b,))
    x = torch.stack([data[i:i+t] for i in ix])
    y = torch.stack([data[i+1:i+t+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [15]:
@torch.no_grad()
def estimate_loss(): # to estimate loss later during the training loop
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [16]:
@dataclasses.dataclass
class GemmaConfig:
    # The number of tokens in the vocabulary.
    vocab_size: int = v # defined earlier when we loaded TinyShakespeare. In Gemma it's 256,128
    
    # The maximum sequence length that this model might ever be used with.
    max_position_embeddings: int = 64 # In Gemma it's 8192
    
    # The number of blocks in the model.
    num_hidden_layers: int = 4 # In Gemma 7b it's 28 and 2b it's 18
    
    # The number of attention heads used in the attention layers of the model.
    num_attention_heads: int = 4 # In Gemma 7b it's 16 and 2b it's 8
    
    # The number of key-value heads for implementing attention.
    num_key_value_heads: int = 1 # In Gemma 7b it's 16 and in 2b it's 1
    # Notice what with Gemma 7b num_attention_heads = num_key_value_heads whereas this is not true for 2b
    # this is because 7b uses regular multi-head attention while 2b uses multi-query attention
    # the difference is that MQA shares single key & value matrices across all heads whereas MHA
    
    # The hidden size of the model.
    hidden_size: int = 128 # In Gemma 7b it's 3072 and in 2b it's 2048
    
    # The dimension of the MLP representations.
    intermediate_size: int = 1024 # In Gemma 7b it's 24576 and in 2b it's 16384
    
    # The number of head dimensions.
    head_dim: int = 32 # In both Gemmas it's 256
    
    # The epsilon used by the rms normalization layers.
    rms_norm_eps: float = 1e-6
    
    # The dtype of the weights.
    dtype: str = 'bfloat16'
    
    # Whether a quantized version of the model is used.
    # quant: bool = False 
    # we won't be providing an option for this in our implementation. All corresponding code for it has been removed
    
    # The path to the model tokenizer.
    #tokenizer: Optional[str] = 'tokenizer/tokenizer.model'
    # instead we'll be using the functions defined above for our character-wise tokenization

    def get_dtype(self) -> Optional[torch.dtype]:
        """Gets the torch dtype from the config dtype string."""
        return _STR_DTYPE_TO_TORCH_DTYPE.get(self.dtype, None)


#def get_config_for_7b() -> GemmaConfig:
    #return GemmaConfig()


#def get_config_for_2b() -> GemmaConfig:
    #return GemmaConfig(
        #num_hidden_layers=18,
        #num_attention_heads=8,
        #num_key_value_heads=1,
        #hidden_size=2048,
        #intermediate_size=16384
    #)


def get_model_config(variant: str) -> GemmaConfig:
    #if variant == '7b':
        #return get_config_for_7b()
    #elif variant == '2b':
        #return get_config_for_2b()
    #return ValueError(f'Invalid variant {variant}. Supported variants are "2b" and "7b"')
    return GemmaConfig()

In [ ]:
class Sampler(nn.Module):

    def __init__(self, vocab_size: int):
        super().__init__()
        self.vocab_size = vocab_size

    @torch.no_grad()
    def forward(
        self,
        embedding: torch.Tensor, # i assume this is [v,d] or [d,v] tensor
        hidden_states: torch.Tensor, # multiple hidden states? I assume single [b,t,d] tensor
        output_positions: torch.Tensor, # NO CLUE WHAT THIS IS
        temperatures: torch.Tensor, # why do we have multiple temperatures?
        top_ps: torch.Tensor, # why multiple top_ps and top_ks? are we able to specify a different one per inference batch or something?
        top_ks: torch.Tensor, # might consider removing the top_k ability later since i think top_p is better
        embedding_bias: Optional[torch.Tensor] = None, # wth is an embedding bias? I assume tensor of size [v] but huh?
    ) -> torch.Tensor:
        
        # Select the last element for each sequence.
        # (batch_size, input_len, hidden_size) -> (batch_size, hidden_size)
        hidden_states = hidden_states.index_select(
            1, output_positions).squeeze(dim=1)
        logits = torch.matmul(hidden_states, embedding.t())
        if embedding_bias is not None:
            logits += embedding_bias

        if temperatures is None:
            return torch.argmax(logits, dim=-1).squeeze(dim=-1)

        # Apply temperature scaling.
        logits.div_(temperatures.unsqueeze(dim=1))

        # Calculate probabilities with softmax.
        probs = torch.softmax(logits, dim=-1, dtype=torch.float)
        probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)

        # Apply top-p, top-k.
        probs_sum = torch.cumsum(probs_sort, dim=-1)
        top_ps_mask = (probs_sum - probs_sort) > top_ps.unsqueeze(dim=1)
        probs_sort = torch.where(top_ps_mask, 0, probs_sort)

        top_ks_mask = torch.arange(probs_idx.shape[-1],
                                   device=probs_idx.device)
        top_ks_mask = top_ks_mask.expand(probs_idx.shape[0], -1)
        top_ks_mask = top_ks_mask >= top_ks.unsqueeze(dim=1)
        probs_sort = torch.where(top_ks_mask, 0, probs_sort)

        # Re-normalization.
        probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
        probs = torch.gather(probs_sort,
                             dim=-1,
                             index=torch.argsort(probs_idx, dim=-1))

        next_token_ids = torch.multinomial(probs,
                                           num_samples=1,
                                           replacement=True).squeeze(dim=-1)
        return next_token_ids